In [14]:
#all imports
import carla #the sim library itself
import random #to pick random spawn point
import cv2 #to work with images from cameras
import numpy as np #in this example to change image representation - re-shaping
import os #to work for something related to joining path stuff
import time#will use for delays later....FPS stuff

In [11]:
# connect to the sim 
client = carla.Client('localhost', 2000)

In [12]:
# Carla now connected

In [3]:
#define environment/world and get possible places to spawn a car
world = client.get_world()
spawn_points = world.get_map().get_spawn_points()

In [4]:
#look for a blueprint of Mini car
vehicle_bp = world.get_blueprint_library().filter('*mini*')

In [5]:
#spawn a car in a random location

start_point = spawn_points[0]
vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)


In [6]:
# move simulator view to the car
spectator = world.get_spectator()
start_point.location.z = start_point.location.z+1 #start_point was used to spawn the car but we move 1m up to avoid being on the floor
spectator.set_transform(start_point)

In [7]:
#send the car off on autopilot - this will leave the spectator
vehicle.set_autopilot(True)

In [8]:
# Define camera parameters
CAMERA_POS_Z = 1.6
CAMERA_POS_X = 0.9
IMAGE_WIDTH = 1280  # Increased width
IMAGE_HEIGHT = 720  # Increased height

# Create camera blueprint
camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', str(IMAGE_WIDTH))
camera_bp.set_attribute('image_size_y', str(IMAGE_HEIGHT))
camera_bp.set_attribute('sensor_tick', '0.0')  # Set sensor tick to 0 to ensure it gets every frame

# Define camera initial transform relative to the vehicle
camera_relative_transform = carla.Transform(carla.Location(z=CAMERA_POS_Z, x=CAMERA_POS_X))

# Spawn camera attached to the vehicle
camera = world.spawn_actor(camera_bp, camera_relative_transform, attach_to=vehicle)

# Callback function to process camera images
def camera_callback(image, data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

# Initialize dictionary to store camera data
camera_data = {'image': np.zeros((IMAGE_HEIGHT, IMAGE_WIDTH, 4), dtype=np.uint8)}

# Set up video writer
video_name = 'captured_video1.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')  # Use MP4V codec
video_out = cv2.VideoWriter(video_name, fourcc, 20.0, (IMAGE_WIDTH, IMAGE_HEIGHT))

# Start listening to camera
camera.listen(lambda image: camera_callback(image, camera_data))

try:
    while True:
        # Display the image
        cv2.imshow('Video Stream', camera_data['image'])

        # Convert RGBA to BGR
        bgr_image = cv2.cvtColor(camera_data['image'], cv2.COLOR_RGBA2RGB)

        # Write frame to video
        video_out.write(bgr_image)

        # Break loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
finally:
    # Clean up
    video_out.release()
    cv2.destroyAllWindows()
